In [1]:
# Experiment 1: Visual Place Recognition with Positive Transformation
# Dataset: GardensPointWalking (day_left vs night_right)
# Feature Descriptors: DELF-HDC
# Descriptor Dimensions: 4096
# Transformation: Positive shift with normalization
# Purpose: Evaluate impact of positive transformation on VPR performance

In [2]:
from scipy.io import loadmat, savemat
from scipy.linalg import orth
import numpy as np
import faiss
from scipy.spatial.distance import cdist
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import gc
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'lib')))

from randn2 import randn2 
import peer_functions as peer
from dot_product import CPUDotProduct, LoihiDotProductSimulationPositive, CPUDotProductPositive
from dot_product import ScaleQuery, ScaleDatabase

import math
from tqdm import tqdm
os.environ['OPENBLAS_NUM_THREADS'] = '1'

c:\Users\21361535\AppData\Local\anaconda3\envs\lava3\lib\site-packages\lava\utils\profiler.py:16: UserWarning: Loihi2HWProfiler could not be imported. Currently no profiler is available.
  warnings.warn("Loihi2HWProfiler could not be imported. "


In [3]:
def calculate_evaluation_metrics(similarity_matrix, ground_truth):
    """
    Calculate evaluation metrics for place recognition results
    
    Args:
        similarity_matrix: Matrix of similarity scores between queries and database
        ground_truth: Ground truth data containing GThard and GTsoft
        
    Returns:
        dict: Dictionary containing recalls, precision, recall curves and AUC
    """
    # Get candidate selections
    ids_pos = peer.directCandSel(similarity_matrix)
    
    # Calculate recall values
    recalls = peer.getRecallAtKVector(ids_pos, ground_truth["GT"])
    
    # Calculate precision-recall curves and AUC
    [R, P] = peer.createPR(-similarity_matrix, 
                          ground_truth['GT']["GThard"][0][0], 
                          ground_truth['GT']["GTsoft"][0][0])
    auc = np.trapz(P, R)
    
    return {
        'recalls': recalls,
        'precision': P,
        'recall': R,
        'auc': auc
    }


In [4]:
D1    = loadmat("../datasets/descriptors/GardensPointWalking/day_left/delf_hdc_4096_ortho_sigma_nx5ny7.mat")['Y']
D2    = loadmat("../datasets/descriptors/GardensPointWalking/night_right/delf_hdc_4096_ortho_sigma_nx5ny7.mat")['Y']

g_truth = loadmat("../datasets/ground_truth/GardensPointWalking/day_left--night_right/gt.mat")

In [5]:
D1_small = D1[0:2,:2].reshape(2,-1)
D2_small = D2[0:2,:2].reshape(2,-1)

print(D1.shape, D1_small.shape)
print(D1_small)
print(D2_small)



(200, 4096) (2, 2)
[[ 30.0405407  -29.0936718 ]
 [ 15.35739326 -46.08417892]]
[[ 32.04243851 -26.73787308]
 [ 13.12259483 -33.11369705]]


In [6]:
dot = CPUDotProduct(D2_small, D2_small)

DD = dot.run()

# metrics = calculate_evaluation_metrics(DD, g_truth)
# print(f"AUC: {metrics['auc']}")

# cpu_auc = metrics['auc']


[[ 32.04243851 -26.73787308]
 [ 13.12259483 -33.11369705]] [[ 32.04243851 -26.73787308]
 [ 13.12259483 -33.11369705]]
mu1 :
[ 22.58251667 -29.92578506]
[[ 0.94763822  0.3193459 ]
 [-0.94763822 -0.3193459 ]] [[ 0.94763822  0.3193459 ]
 [-0.94763822 -0.3193459 ]]
scaled_dbVectors :
[[ 0.94763822  0.3193459 ]
 [-0.94763822 -0.3193459 ]]
scaled_queryVectors :
[[ 0.94763822  0.3193459 ]
 [-0.94763822 -0.3193459 ]]


In [7]:
DD

array([[0., 2.],
       [2., 0.]])

## Positive Transformation

In [8]:
constant = 1

dot = CPUDotProductPositive(D1_small, D2_small, constant)


In [9]:
DD = dot.run()

mu1 :
[ 22.69896698 -37.58892536]
transform_dbVectors :
[[0.68549215 0.72808001]
 [0.81801889 0.57519135]]
transform_queryVectors :
[[0.68495069 0.72858942]
 [0.06592738 0.99782442]]


In [10]:
DD

array([[2.76338125e-07, 2.06190564e-02],
       [2.28311278e-01, 3.72130177e-01]])

In [11]:
# constant = 0

dot = LoihiDotProductSimulationPositive(D1_small, D2_small, constant)

In [12]:
DD = dot.run()

mu1 :
[ 22.69896698 -37.58892536]
transform_dbVectors :
[[34.27460773 36.40400067]
 [40.90094459 28.75956765]]
transform_queryVectors :
[[34.24753473 36.42947111]
 [ 3.29636902 49.89122118]]


100%|██████████| 2/2 [00:00<00:00, 21.79it/s]


init_v :
[-14.752465271890884, -12.570528894218633]
init_v :
[-45.703630980326786, 0.8912211845544888]


In [13]:
DD

array([[-2509.53125 , -2495.65625 ],
       [-1923.46875 , -1603.484375]])